In [101]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from google.colab import drive
import statsmodels.api as sm
import numpy as np
import scipy
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:

ETF=pd.read_csv('/content/drive/Shareddrives/Capstone B1/Spinnaker Data/US Sector Inst ETF.csv')
Institutional=pd.read_csv('/content/drive/Shareddrives/Capstone B1/Spinnaker Data/US Sector Institutional MF.csv')
Retail=pd.read_csv('/content/drive/Shareddrives/Capstone B1/Spinnaker Data/US Sector Retail MF.csv')
sp500=pd.read_csv('/content/drive/Shareddrives/Capstone B1/Spinnaker Data/sp500index.csv')

In [103]:
# Get rolling average of a specific column for differect industries
# table is the data table you need to calculate the rolling average
# window_length means how many rows you want to include in one average calculation
# on what is the variable you what to calculate the rolling average
# new_column_name is the name of the new column of the rolling average
def get_rolling_average(table,window_length,on_what,new_column_name='rollingaverage'):
  result=pd.DataFrame(columns=table.columns)
  for i in table['AssetClass'].unique():
    Retailrolling=table[table['AssetClass']==i]
    Retailrolling[new_column_name]=Retailrolling.rolling(window=window_length, center=False, on=on_what, axis=0, closed=None)[on_what].mean()
    result=pd.concat([result,Retailrolling])
  return result

# generate the label varibale based on the threshold
# it would return 1 for those values higher than upper bound, -1 for those lower than lower threshold, and 0 for others
def get_label(table_orginal,upper=0.05,lower=-0.05):
  def get_label_inner_function(change):
    if change>upper:
      return 1
    elif change<lower:
      return -1
    elif np.isnan(change):
      return np.nan
    else:
      return 0
  
  table=table_orginal
  table['change']=table['rollingaverage']/table['AssetsEnd']-1
  table['label']=table['change'].apply(get_label_inner_function)
  table_orginal['label']=table['label']
  return table_orginal

# clean the report date data to an integer
def clean_reportdate(s):
  s=s.replace(' 12:00:00 AM', '')
  s_list=s.split('/')
  result=str(s_list[2])+str(s_list[0]).rjust(2,'0')+str(s_list[1]).rjust(2,'0')
  return int(result)

# clean the sp500 data to the same format so that we could merge it with our data
def clean_date_sp599(s):
  import calendar
  s_list=s.split('-')
  if int(s_list[2])<=18:
    s_list[2]='20'+s_list[2]
  else:
    s_list[2]='19'+s_list[2]
  return int(s_list[2]+str(list(calendar.month_abbr).index(s_list[1])).rjust(2,'0')+s_list[0])

# calculate the weekly change of sp500 index
def get_sp_change(table):
  result=pd.DataFrame(columns=table.columns)
  for i in table['AssetClass'].unique():
    spchange=table[table['AssetClass']==i]
    spchange['sp_change']=(spchange['close']/(spchange['close'].shift(periods=-1,axis=0))-1)*100
    result=pd.concat([result,spchange])
  return result

# calculate the excess return based on the idea of CAPM and using regression for each industry
def get_excess_change(table):
  result=pd.DataFrame(columns=table.columns)
  lm_result=dict()
  for i in table['AssetClass'].unique():
    subtable=table[table['AssetClass']==i].dropna(axis=0)
    x=np.array(subtable['sp_change'])
    X = sm.add_constant(x)
    y=np.array(subtable['PortfolioChangePct'])
    lm = sm.OLS(y, X).fit()
    lm_result[i]=lm
    LinearRegression()
    subtable['excess_change']=subtable[['PortfolioChangePct']]-LinearRegression().fit(subtable[['sp_change']],subtable[['PortfolioChangePct']]).predict(subtable[['PortfolioChangePct']])
    result=pd.concat([result,subtable])
  return result,lm_result

def merge_sp(table):
  table['ReportDate']=table['ReportDate'].apply(clean_reportdate)
  sp500=pd.read_csv('/content/drive/Shareddrives/Capstone B1/Spinnaker Data/sp500index.csv')
  sp500['date']=sp500['date'].apply(clean_date_sp599)
  table=table.merge(sp500,left_on='ReportDate',right_on='date',how='left')
  return table

def get_t_test_label(table):
  table=merge_sp(table)
  table=get_sp_change(table)
  table,lm_result=get_excess_change(table)
  table['t_test']=table.rolling(window=25, center=False, on='excess_change', axis=0, closed=None)['excess_change'].apply(t_test)
  return table

def t_test(data):
  test_result=scipy.stats.ttest_1samp(data, popmean=0, nan_policy='omit')
  return test_result[1]

In [ ]:
Retail_assetend_rolling=get_rolling_average(Retail,25,'AssetsEnd')
Retail_assetend_rolling=get_label(Retail_assetend_rolling)
Retail_assetend_rolling.to_csv('Retail_assetend_rolling.csv')

Institutional_assetend_rolling=get_rolling_average(Institutional,25,'AssetsEnd')
Institutional_assetend_rolling=get_label(Institutional_assetend_rolling)
Institutional_assetend_rolling.to_csv('Institutional_assetend_rolling.csv')

ETF_assetend_rolling=get_rolling_average(ETF,25,'AssetsEnd')
ETF_assetend_rolling=get_label(ETF_assetend_rolling)
ETF_assetend_rolling.to_csv('ETF_assetend_rolling.csv')

In [ ]:
Retail_excess=get_t_test_label(Retail)
Retail_excess.to_csv('Retail_excess_t_test.csv')

Institutional_excess=get_t_test_label(Institutional)
Institutional_excess.to_csv('Institutional_excess_t_test.csv')

ETF_excess=get_t_test_label(ETF)
ETF_excess.to_csv('ETF_excess_t_test.csv')